# Lista de tablas

## Librerias

In [1]:
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
    "account": credentials["ACCOUNT_SNOWFLAKE"],
    "user": credentials["USER_SNOWFLAKE"],
    "password": credentials["PASSWORD_SNOWFLAKE"],
    "role": credentials["ROLE_SNOWFLAKE"]
}

# Paso 4: Crear una sesión de Snowflake usando los parámetros de conexión
session_activa = Session.builder.configs(connection_parameters).create()

# Paso 5: Verificar la conexión
print(f"Sesión activa: {session_activa}")

Sesión activa: <snowflake.snowpark.session.Session: account="icb13635", role="ACCOUNTADMIN", database=None, schema=None, warehouse="COMPUTE_WH">


## 1. Exportaciones

Crear consulta general que pueda tener los siguientes parámetros:

- Agrupación geográfica: 
    - Contienente
    - Zona geográfica
    - País
    - Departamento
- Agrupación administrativa:
    - HUB
- Acuerdo comercial: 
    - TLCS: atado a TIPO_ACUERDO
- Posición arancelaria: 
    - TIPO
- Agrupación de fechas:
    - YEAR

Crear consulta para la cuenta de empresas exportadoras y datos de empresas exportadoras que pueda tener los siguientes parámetros:

- Agrupación geográfica: 
    - Contienente
    - Zona geográfica
    - País
    - Departamento (más exporta)
- Agrupación administrativa:
    - HUB
- Acuerdo comercial: 
    - TLCS: atado a TIPO_ACUERDO
- Agrupación de fechas:
    - YEAR

#### Consulta general

In [5]:
def get_data_exportaciones(session, continentes=None, zonas_geograficas=None, paises=None, departamentos=None, hubs=None,tlcs=None, tipo_tlcss=None, tipos=None, years=None):
    """
    Extrae datos de exportaciones desde Snowflake aplicando filtros específicos y agrupa los resultados.

    Parámetros:
    session (snowflake.snowpark.Session): Sesión activa en Snowflake.
    continentes (list): Lista de continentes a filtrar.
    zonas_geograficas (list): Lista de zonas geográficas a filtrar.
    paises (list): Lista de países a filtrar.
    departamentos (list): Lista de departamentos a filtrar.
    hubs (list): Lista de hubs a filtrar.
    tlcs (list): Lista de tratados de libre comercio a filtrar.
    tipo_tlcss (list): Lista de tipos de acuerdos comerciales a filtrar.
    tipos (list): Lista de tipos de posición arancelaria a filtrar.
    years (list): Lista de años a filtrar.

    Pasos del proceso:
    1. Verificar que los parámetros son listas o None.
    2. Construir la consulta SQL base.
    3. Añadir condiciones a la consulta SQL según los parámetros proporcionados.
    4. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas.
    5. Devolver el DataFrame resultante.

    Retorna:
    DataFrame: DataFrame con los datos de exportaciones filtrados y agrupados.
    """
    
    # 1. Verificar que los parámetros son listas o None
    for param in [tipos, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, years]:
        if param is not None and not isinstance(param, list):
            raise ValueError("Todos los parámetros deben ser listas o None")
        
    # 2. Construir la consulta SQL base
    query = """
    SELECT A.TIPO,
        A.CADENA,
        A.SECTOR,
        A.SUBSECTOR,
        A.PAIS_DESTINO,
        A.HUB,
        A.CONTINENTE,
        A.ZONA_GEOGRAFICA,
        A.TLCS,
        A.TIPO_ACUERDO,
        A.DEPARTAMENTO_ORIGEN,
        A.MEDIO_TRANSPORTE,
        A.CADENA_FRIO,
        A.TIPO_ESTRELLA,
        A.CADENA_ESTRELLA,
        A.SECTOR_ESTRELLA,
        A.SUBSECTOR_ESTRELLA,
        A.DPTO_MAS_EXPORTA_ESTRELLA,
        A.YEAR,
        SUM(A.VALOR_USD) AS VALOR_USD, 
        SUM(A.PESO_KG) AS PESO_KG_NETO
    FROM DOCUMENTOS_COLOMBIA.EXPORTACIONES.BASE_EXPORTACIONES AS A
    WHERE 1=1
    """
    
    # 3. Añadir condiciones a la consulta SQL según los parámetros proporcionados

    # Agrupación geográfica: 
    if continentes:
        query += f""" AND A.CONTINENTE IN ({','.join([f"'{continente}'" for continente in continentes])})"""
    if zonas_geograficas:
        query += f""" AND A.ZONA_GEOGRAFICA IN ({','.join([f"'{zona}'" for zona in zonas_geograficas])})"""
    if paises:
        query += f""" AND A.PAIS_DESTINO IN ({','.join([f"'{pais}'" for pais in paises])})"""
    if departamentos:
        query += f""" AND A.DEPARTAMENTO_ORIGEN IN ({','.join([f"'{departamento}'" for departamento in departamentos])})"""
        
    # Agrupación administrativa:
    if hubs:
        query += f""" AND A.HUB IN ({','.join([f"'{hub}'" for hub in hubs])})"""
    
    # Acuerdo comercial:
    if tlcs:
        query += f""" AND A.TLCS IN ({','.join([f"'{tlc}'" for tlc in tlcs])})"""
    if tipo_tlcss:
        query += f""" AND A.TIPO_ACUERDO IN ({','.join([f"'{tipo_tlcs}'" for tipo_tlcs in tipo_tlcss])})"""
    
    # Posición arancelaria:
    if tipos:
        query += f""" AND A.TIPO IN ({','.join([f"'{tipo}'" for tipo in tipos])})"""

    # Agrupación de fechas (dos fechas siempre):
    if years:
        query += f""" AND A.YEAR IN ({','.join([f"'{year}'" for year in years])})"""
    
    # Agregar group by 
    query += """
    GROUP BY A.TIPO,
        A.CADENA,
        A.SECTOR,
        A.SUBSECTOR,
        A.PAIS_DESTINO,
        A.HUB,
        A.CONTINENTE,
        A.ZONA_GEOGRAFICA,
        A.TLCS,
        A.TIPO_ACUERDO,
        A.DEPARTAMENTO_ORIGEN,
        A.MEDIO_TRANSPORTE,
        A.CADENA_FRIO,
        A.TIPO_ESTRELLA,
        A.CADENA_ESTRELLA,
        A.SECTOR_ESTRELLA,
        A.SUBSECTOR_ESTRELLA,
        A.DPTO_MAS_EXPORTA_ESTRELLA,
        A.YEAR
    """
    
    # 4. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas
    data = pd.DataFrame(session.sql(query).collect())
    
    # 5. Devolver el DataFrame resultante
    return data


#### Consulta cuenta de empresas

In [6]:
def get_data_exportaciones_numero_empresas(session, continentes=None, zonas_geograficas=None, paises=None, departamentos=None, hubs=None, tlcs=None, tipo_tlcss=None, tipos=None, years=None, umbral=10000):
    """
    Extrae datos de exportaciones desde Snowflake aplicando filtros específicos y cuenta el número de empresas únicas que superan un umbral de exportación.

    Parámetros:
    session (snowflake.snowpark.Session): Sesión activa en Snowflake.
    continentes (list): Lista de continentes a filtrar.
    zonas_geograficas (list): Lista de zonas geográficas a filtrar.
    paises (list): Lista de países a filtrar.
    departamentos (list): Lista de departamentos a filtrar.
    hubs (list): Lista de hubs a filtrar.
    tlcs (list): Lista de tratados de libre comercio a filtrar.
    tipo_tlcss (list): Lista de tipos de acuerdos comerciales a filtrar.
    tipos (list): Lista de tipos de posición arancelaria a filtrar.
    years (list): Lista de años a filtrar.
    umbral (int): Umbral mínimo de exportación en USD para considerar una empresa.

    Pasos del proceso:
    1. Verificar que los parámetros son listas o None.
    2. Construir la consulta SQL base.
    3. Añadir condiciones a la consulta SQL según los parámetros proporcionados.
    4. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas.
    5. Filtrar el DataFrame para empresas que superan el umbral de exportación.
    6. Contar el número de empresas únicas.
    7. Devolver el número de empresas únicas.

    Retorna:
    int: Número de empresas únicas que superan el umbral de exportación.
    """
    
    # 1. Verificar que los parámetros son listas o None
    for param in [continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos, years]:
        if param is not None and not isinstance(param, list):
            raise ValueError("Todos los parámetros deben ser listas o None")
        
    # 2. Construir la consulta SQL base
    query = """
    SELECT A.NIT_EXPORTADOR,
        A.RAZON_SOCIAL,
        A.SECTOR_ESTRELLA,
        A.YEAR, 
        SUM(A.VALOR_USD) AS VALOR_USD
    FROM DOCUMENTOS_COLOMBIA.EXPORTACIONES.BASE_EXPORTACIONES AS A
    WHERE A.TIPO = 'No Mineras' 
        AND A.TIPO_ESTRELLA = 'No Mineras' 
        AND A.CADENA_ESTRELLA IN ('Agroalimentos', 'Industrias 4.0', 'Metalmecánica y Otras Industrias', 'Químicos y Ciencias de la Vida', 'Sistema Moda')
        AND A.NIT_EXPORTADOR NOT IN ('-1')
    """

    # 3. Añadir condiciones a la consulta SQL según los parámetros proporcionados

    # Agrupación geográfica: 
    if continentes:
        query += f""" AND A.CONTINENTE IN ({','.join([f"'{continente}'" for continente in continentes])})"""
    if zonas_geograficas:
        query += f""" AND A.ZONA_GEOGRAFICA IN ({','.join([f"'{zona}'" for zona in zonas_geograficas])})"""
    if paises:
        query += f""" AND A.PAIS_DESTINO IN ({','.join([f"'{pais}'" for pais in paises])})"""
    if departamentos:
        query += f""" AND A.DPTO_MAS_EXPORTA_ESTRELLA IN ({','.join([f"'{departamento}'" for departamento in departamentos])})"""
        
    # Agrupación administrativa:
    if hubs:
        query += f""" AND A.HUB IN ({','.join([f"'{hub}'" for hub in hubs])})"""
    
    # Acuerdo comercial:
    if tlcs:
        query += f""" AND A.TLCS IN ({','.join([f"'{tlc}'" for tlc in tlcs])})"""
    if tipo_tlcss:
        query += f""" AND A.TIPO_ACUERDO IN ({','.join([f"'{tipo_tlcs}'" for tipo_tlcs in tipo_tlcss])})"""
    
    # Posición arancelaria:
    if tipos:
        query += f""" AND A.TIPO IN ({','.join([f"'{tipo}'" for tipo in tipos])})"""

    # Agrupación de fechas (dos fechas siempre):
    if years:
        query += f""" AND A.YEAR IN ({','.join([f"'{year}'" for year in years])})"""
    
    # 4. Añadir group by
    query += """
    GROUP BY A.NIT_EXPORTADOR,
        A.RAZON_SOCIAL,
        A.SECTOR_ESTRELLA,
        A.YEAR;
    """

    # 5. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas
    data = pd.DataFrame(session.sql(query).collect())

    # 6. Filtrar el DataFrame para empresas que superan el umbral de exportación
    dfcuenta = data[data['VALOR_USD'] > umbral]

    # 7. Contar el número de empresas únicas
    empresas_unicas = dfcuenta['NIT_EXPORTADOR'].nunique()

    # 8. Devolver el número de empresas únicas
    return empresas_unicas

#### Consulta de datos de empresas

In [7]:
def get_data_exportaciones_empresas(session, continentes=None,zonas_geograficas=None, paises=None, departamentos=None, hubs=None, tlcs=None, tipo_tlcss=None, tipos=None, years=None):
    """
    Extrae datos de exportaciones desde Snowflake aplicando filtros específicos y obtiene información de empresas y totales de exportación.

    Parámetros:
    session (snowflake.snowpark.Session): Sesión activa en Snowflake.
    continentes (list): Lista de continentes a filtrar.
    zonas_geograficas (list): Lista de zonas geográficas a filtrar.
    paises (list): Lista de países a filtrar.
    departamentos (list): Lista de departamentos a filtrar.
    hubs (list): Lista de hubs a filtrar.
    tlcs (list): Lista de tratados de libre comercio a filtrar.
    tipo_tlcss (list): Lista de tipos de acuerdos comerciales a filtrar.
    tipos (list): Lista de tipos de posición arancelaria a filtrar.
    years (list): Lista de años a filtrar.
    umbral (int): Umbral mínimo de exportación en USD para considerar una empresa.

    Pasos del proceso:
    1. Verificar que los parámetros son listas o None.
    2. Construir la consulta SQL para datos de empresas.
    3. Añadir condiciones a la consulta SQL según los parámetros proporcionados.
    4. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas.
    5. Construir la consulta SQL para datos totales.
    6. Añadir condiciones a la consulta SQL según los parámetros proporcionados.
    7. Ejecutar la consulta SQL y convertir los resultados en un DataFrame de pandas.
    8. Devolver los DataFrames de empresas y totales.

    Retorna:
    tuple: DataFrames con los datos de empresas y los totales de exportación.
    """
    
    # 1. Verificar que los parámetros son listas o None
    for param in [continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos, years]:
        if param is not None and not isinstance(param, list):
            raise ValueError("Todos los parámetros deben ser listas o None")
        
    # 2. Construir la consulta SQL de datos de empresas
    query1 = """
    SELECT A.NIT_EXPORTADOR,
        A.RAZON_SOCIAL,
        A.SECTOR_ESTRELLA,
        A.YEAR, 
        SUM(A.Valor_USD) AS VALOR_USD
    FROM DOCUMENTOS_COLOMBIA.EXPORTACIONES.BASE_EXPORTACIONES AS A
    WHERE A.TIPO = 'No Mineras' 
        AND A.TIPO_ESTRELLA = 'No Mineras' 
        AND A.NIT_EXPORTADOR NOT IN ('-1')
    """
    
    # 3. Añadir condiciones a la consulta SQL según los parámetros proporcionados

    # Agrupación geográfica: 
    if continentes:
        query1 += f""" AND A.CONTINENTE IN ({','.join([f"'{continente}'" for continente in continentes])})"""
    if zonas_geograficas:
        query1 += f""" AND A.ZONA_GEOGRAFICA IN ({','.join([f"'{zona}'" for zona in zonas_geograficas])})"""
    if paises:
        query1 += f""" AND A.PAIS_DESTINO IN ({','.join([f"'{pais}'" for pais in paises])})"""
    if departamentos:
        query1 += f""" AND A.DPTO_MAS_EXPORTA_ESTRELLA IN ({','.join([f"'{departamento}'" for departamento in departamentos])})"""
        
    # Agrupación administrativa:
    if hubs:
        query1 += f""" AND A.HUB IN ({','.join([f"'{hub}'" for hub in hubs])})"""
    
    # Acuerdo comercial:
    if tlcs:
        query1 += f""" AND A.TLCS IN ({','.join([f"'{tlc}'" for tlc in tlcs])})"""
    if tipo_tlcss:
        query1 += f""" AND A.TIPO_ACUERDO IN ({','.join([f"'{tipo_tlcs}'" for tipo_tlcs in tipo_tlcss])})"""
    
    # Posición arancelaria:
    if tipos:
        query1 += f""" AND A.TIPO IN ({','.join([f"'{tipo}'" for tipo in tipos])})"""

    # Agrupación de fechas (dos fechas siempre):
    if years:
        query1 += f""" AND A.YEAR IN ({','.join([f"'{year}'" for year in years])})"""    
    
    # Añadir group by
    query1 += """
    GROUP BY A.NIT_EXPORTADOR,
        A.RAZON_SOCIAL,
        A.SECTOR_ESTRELLA,
        A.YEAR;
    """

    # 4. Construir la consulta SQL del total
    query2 = """
    SELECT A.YEAR, 
        SUM(A.Valor_USD) AS VALOR_USD
    FROM DOCUMENTOS_COLOMBIA.EXPORTACIONES.BASE_EXPORTACIONES AS A
    WHERE A.TIPO = 'No Mineras' 
    """
    
    # 5. Añadir condiciones a la consulta SQL según los parámetros proporcionados

    # Agrupación geográfica: 
    if continentes:
        query2 += f""" AND A.CONTINENTE IN ({','.join([f"'{continente}'" for continente in continentes])})"""
    if zonas_geograficas:
        query2 += f""" AND A.ZONA_GEOGRAFICA IN ({','.join([f"'{zona}'" for zona in zonas_geograficas])})"""
    if paises:
        query2 += f""" AND A.PAIS_DESTINO IN ({','.join([f"'{pais}'" for pais in paises])})"""
    if departamentos:
        query2 += f""" AND A.DPTO_MAS_EXPORTA_ESTRELLA IN ({','.join([f"'{departamento}'" for departamento in departamentos])})"""
        
    # Agrupación administrativa:
    if hubs:
        query2 += f""" AND A.HUB IN ({','.join([f"'{hub}'" for hub in hubs])})"""
    
    # Acuerdo comercial:
    if tlcs:
        query2 += f""" AND A.TLCS IN ({','.join([f"'{tlc}'" for tlc in tlcs])})"""
    if tipo_tlcss:
        query2 += f""" AND A.TIPO_ACUERDO IN ({','.join([f"'{tipo_tlcs}'" for tipo_tlcs in tipo_tlcss])})"""
    
    # Posición arancelaria:
    if tipos:
        query2 += f""" AND A.TIPO IN ({','.join([f"'{tipo}'" for tipo in tipos])})"""

    # Agrupación de fechas (dos fechas siempre):
    if years:
        query2 += f""" AND A.YEAR IN ({','.join([f"'{year}'" for year in years])})"""    
    
    # Añadir group by
    query2 += """
    GROUP BY A.YEAR;
    """
    
    # 6. Ejecutar la consulta SQL de empresas
    df_empresas = pd.DataFrame(session.sql(query1).collect())

    # 7. Ejecutar la consulta SQL de totales
    df_totales = pd.DataFrame(session.sql(query2).collect())
    
    # 8. Devolver los DataFrames de empresas y totales
    return df_empresas, df_totales

#### Funciones para generar tablas

In [8]:
def transform_year_column_name(col_name):
    """
    Transforma el nombre de una columna de año para un formato más amigable.
    
    Parámetros:
    col_name (str): Nombre de la columna que contiene el año y el período.

    Pasos del proceso:
    1. Verifica si el nombre de la columna contiene paréntesis.
    2. Si contiene paréntesis, separa el año y el período.
    3. Formatea el nombre de la columna en el nuevo formato 'período año'.
    4. Si no contiene paréntesis, retorna el nombre original de la columna.

    Retorna:
    str: El nombre de la columna transformado si contiene paréntesis, de lo contrario, el nombre original.
    """
    
    # Verificar si el nombre de la columna contiene paréntesis
    if '(' in col_name and ')' in col_name:
        # Separar el año y el período
        year, period = col_name.split('(')
        period = period.replace(')', '')
        
        # Formatear el nombre de la columna en el nuevo formato 'período año'
        return f'{period} {year.strip()}'
    
    # Si no contiene paréntesis, retornar el nombre original de la columna
    return col_name

In [9]:
def transform_year(col_name):
    """
    Extrae el año de un nombre de columna que contiene el año y el período.
    
    Parámetros:
    col_name (str): Nombre de la columna que contiene el año y el período.

    Pasos del proceso:
    1. Verifica si el nombre de la columna contiene paréntesis.
    2. Si contiene paréntesis, separa el año y el período.
    3. Retorna el año extraído.
    4. Si no contiene paréntesis, retorna el nombre original.

    Retorna:
    str: El año extraído si contiene paréntesis, de lo contrario, el nombre original.
    """
    
    # Verificar si el nombre de la columna contiene paréntesis
    if '(' in col_name and ')' in col_name:
        # Separar el año y el período
        year, period = col_name.split('(')
        return year.strip()
    
    # Si no contiene paréntesis, retornar el nombre original de la columna
    return f'{col_name}'

In [10]:
def generar_tabla_resumen(df, categoria, valor, top_n = None):

    """
    Genera una tabla resumen a partir de un DataFrame de exportaciones, agregando columnas de variación porcentual y participación.
    
    Parámetros:
    df (DataFrame): DataFrame proveniente de la función get_data_exportaciones().
    categoria (str): Variable categórica para la cual se generará la tabla de resumen (e.g., 'PAIS_DESTINO', 'SECTOR').
    valor (str): Variable de valor a agregar ('VALOR_USD' o PESO_KG_NETO).
    top_n: Número de categorías top a filtrar.Por defecto no se filtran.

    Pasos del proceso:
    1. Crear una tabla pivote sumando el valor por categoría y año.
    2. Ordenar las columnas de año para manejar períodos correctamente.
    3. Filtrar el top 5 y agrupar los demás en 'Otros'.
    4. Agregar una fila con los totales generales.
    5. Calcular la variación entre el penúltimo y el último año.
    6. Calcular la participación en el último año.
    7. Reordenar las columnas para tener los años, la variación y la participación.
    8. Ordenar categorías por valor del último año, excepto 'Otros' y 'Total'.
    9. Formatear los valores y porcentajes.
    10. Asegurar que los nombres de las columnas no tengan múltiples niveles.
    11. Renombrar la columna de categoría.
    12. Transformar los nombres de las columnas de año y agrega formatos decimales en español.

    Retorna:
    DataFrame: Tabla resumen con las categorías ordenadas, variación y participación calculadas y formateadas.
    """

    # 1. Crear tabla pivote sumando el valor por categoría y año
    pivot_table = df.pivot_table(values=valor, index=categoria, columns='YEAR', aggfunc='sum', fill_value=0)

    # Ordenar las columnas de año para manejar períodos correctamente
    ordered_years = sorted(pivot_table.columns, key=lambda x: (int(x.split('(')[0]) if '(' in x else int(x)))
    pivot_table = pivot_table[ordered_years]

    # 2. Filtrar el top_n y agrupar los demás en 'Otros'
    if top_n:
        # Obtener las 5 categorías principales basadas en el valor del último año disponible
        topn = pivot_table.nlargest(top_n, ordered_years[-1])
        # Agrupar las demás categorías bajo 'Otros'
        others = pivot_table.drop(topn.index)
        others_sum = others.sum().to_frame().T
        others_sum[categoria] = 'Otros'
        pivot_table = pd.concat([topn, others_sum.set_index(categoria)])

    
    # 3. Agregar fila con los totales generales
    totals = pivot_table.sum().to_frame().T
    totals[categoria] = 'Total'
    pivot_table = pd.concat([pivot_table, totals.set_index(categoria)])

    # 4. Calcular la variación entre años
    year_prev = ordered_years[-2]
    year_curr = ordered_years[-1]
    pivot_table['Variación (%)'] = (pivot_table[year_curr] - pivot_table[year_prev]) / pivot_table[year_prev] * 100

    # 5. Calcular la participación en el último año
    total_last_year = pivot_table.loc['Total', year_curr]
    pivot_table[f'Participación {year_curr} (%)'] = (pivot_table[year_curr] / total_last_year) * 100

    # 6. Reordenar las columnas para tener los años, la variación y la participación
    ordered_columns = ordered_years + ['Variación (%)', f'Participación {year_curr} (%)']
    pivot_table = pivot_table[ordered_columns]

    # 7. Ordenar categorías por valor del último año, excepto 'Otros' y 'Total'
    pivot_table = pivot_table.reset_index()
    pivot_table = pivot_table.sort_values(by=year_curr, ascending=False)
    if 'Otros' in pivot_table[categoria].values:
        otros_row = pivot_table[pivot_table[categoria] == 'Otros']
        pivot_table = pivot_table[pivot_table[categoria] != 'Otros']
        pivot_table = pd.concat([pivot_table, otros_row])
    total_row = pivot_table[pivot_table[categoria] == 'Total']
    pivot_table = pivot_table[pivot_table[categoria] != 'Total']
    pivot_table = pd.concat([pivot_table, total_row])
    pivot_table = pivot_table.set_index(categoria)

    # 8. Formatear los valores y porcentajes
    pivot_table[ordered_years] = pivot_table[ordered_years].applymap('{:,.0f}'.format)
    pivot_table[['Variación (%)', f'Participación {year_curr} (%)']] = pivot_table[['Variación (%)', f'Participación {year_curr} (%)']].applymap('{:.1f}%'.format)

    # 9. Asegurar que los nombres de las columnas no tengan múltiples niveles
    pivot_table.columns.name = None
    pivot_table.reset_index(inplace=True)

    # 10. Renombrar la columna de categoría
    column_names_dict = {
        'TIPO': 'Tipo de exportación',
        'CADENA': 'Cadena',
        'SECTOR': 'Sector',
        'SUBSECTOR': 'Subsector',
        'PAIS_DESTINO': 'País destino',
        'HUB': 'HUB',
        'CONTINENTE': 'Continente',
        'ZONA_GEOGRAFICA': 'Zona geográfica',
        'TLCS': 'Tratados de Libre Comercio',
        'DEPARTAMENTO_ORIGEN': 'Departamento de origen',
        'MEDIO_TRANSPORTE': 'Medio de transporte',
        'YEAR': 'Año'}
    if categoria in column_names_dict:
        pivot_table.rename(columns={categoria: column_names_dict[categoria]}, inplace=True)

    # 11. Transformar los nombres de las columnas de año
    new_columns = {}
    for col in pivot_table.columns:
        if 'Participación' in col:
            new_columns[col] = f'Participación {transform_year(year_curr)} (%)'
        else:
            new_columns[col] = col
    pivot_table.rename(columns=new_columns, inplace=True)  

    # 12. Agregar USD FOB a las columnas de años
    if valor == 'VALOR_USD':
        pivot_table.rename(columns={pivot_table.columns[1]: f'{transform_year_column_name(pivot_table.columns[1])} (USD FOB)'}, inplace=True)
        pivot_table.rename(columns={pivot_table.columns[2]: f'{transform_year_column_name(pivot_table.columns[2])} (USD FOB)'}, inplace=True)
    if valor == 'PESO_KG_NETO':
        pivot_table.rename(columns={pivot_table.columns[1]: f'{transform_year_column_name(pivot_table.columns[1])} (KG NETO)'}, inplace=True)
        pivot_table.rename(columns={pivot_table.columns[2]: f'{transform_year_column_name(pivot_table.columns[2])} (KG NETO)'}, inplace=True)

    # Convertir separadores de decimales y miles
    pivot_table = pivot_table.astype(str)
    for col in pivot_table.columns[1:]:
        pivot_table[col] = pivot_table[col].apply(lambda x: x.replace(',', 'X').replace('.', ',').replace('X', '.'))
    
    # 13. Resultado
    return pivot_table

In [11]:
def generar_tabla_empresas(df_empresas, df_totales, year1, year2):
    """
    Genera un resumen completo de exportaciones por NIT, incluyendo los valores de exportación de los años especificados,
    la variación porcentual entre esos años y la participación en el último año. También agrega filas para 'Otros' 
    y 'Total' en la tabla final.

    Parámetros:
    df_empresas (DataFrame): DataFrame con los datos de exportaciones por empresa.
    df_totales (DataFrame): DataFrame con los totales de exportación por año.
    year1 (int): Año inicial para el cálculo de variaciones.
    year2 (int): Año final para el cálculo de variaciones.

    Pasos del proceso:
    1. Crear una tabla pivote con los valores de exportación por empresa y año.
    2. Asegurarse de que los años especificados existen en la tabla pivote.
    3. Calcular la participación porcentual del último año.
    4. Calcular la variación porcentual entre los años especificados.
    5. Resetear el índice de la tabla pivote para facilitar su manejo.
    6. Seleccionar las columnas deseadas para el resumen.
    7. Filtrar el top 5 de empresas y agrupar las demás en 'Otros'.
    8. Crear la fila de totales de exportación.
    9. Calcular los valores de 'Otros' restando los del top 5 a los totales.
    10. Combinar las filas de top 5, 'Otros' y 'Total' en el DataFrame final.
    11. Renombrar las columnas y formatear los valores y porcentajes.
    
    Retorna:
    DataFrame: DataFrame con el resumen completo de exportaciones.
    """
    
    # 1. Crear la tabla pivote con los valores de exportación por empresa y año
    df_pivot = df_empresas.pivot_table(values='VALOR_USD', index=['NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA'], columns='YEAR', aggfunc='sum', fill_value=0)
    
    # 2. Asegurarse de que los años especificados existen en la tabla pivote
    if year1 not in df_pivot.columns:
        df_pivot[year1] = 0
    if year2 not in df_pivot.columns:
        df_pivot[year2] = 0
    
    # 3. Calcular la participación porcentual del último año
    total_year2 = df_pivot[year2].sum()
    df_pivot['Participación (%)'] = (df_pivot[year2] / total_year2) * 100

    # 4. Calcular la variación porcentual entre los años especificados
    df_pivot['Variación (%)'] = ((df_pivot[year2] - df_pivot[year1]) / df_pivot[year1].replace(0, np.nan)) * 100
    
    # 5. Resetear el índice de la tabla pivote para facilitar su manejo
    df_pivot.reset_index(inplace=True)
    
    # 6. Seleccionar las columnas deseadas para el resumen
    df_pivot = df_pivot[['NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', year1, year2, 'Variación (%)', 'Participación (%)']]
    
    # 7. Filtrar el top 5 de empresas y agrupar las demás en 'Otros'
    top5 = df_pivot.nlargest(5, year2)
    
    # 8. Crear la fila de totales de exportación
    total_row = df_totales[df_totales['YEAR'] == year2].copy()
    total_row['NIT_EXPORTADOR'] = 'Total'
    total_row['RAZON_SOCIAL'] = ''
    total_row['SECTOR_ESTRELLA'] = ''
    total_row['Variación (%)'] = 100.0
    total_row['Participación (%)'] = 100.0
    
    # Asegurarse de que los años existen en el DataFrame de totales
    if year1 not in df_totales['YEAR'].values:
        total_row[year1] = 0
    else:
        total_row[year1] = df_totales[df_totales['YEAR'] == year1]['VALOR_USD'].values[0]
    
    total_row[year2] = df_totales[df_totales['YEAR'] == year2]['VALOR_USD'].values[0]
    
    total_row = total_row[['NIT_EXPORTADOR', 'RAZON_SOCIAL', 'SECTOR_ESTRELLA', year1, year2, 'Variación (%)', 'Participación (%)']]
    
    # Convertir total_row en DataFrame si es una serie
    if isinstance(total_row, pd.Series):
        total_row = total_row.to_frame().T
    
    # 9. Calcular los valores de 'Otros' restando los del top 5 a los totales
    others_sum = total_row.copy()
    others_sum.iloc[0, others_sum.columns.get_loc(year1)] -= top5[year1].sum()
    others_sum.iloc[0, others_sum.columns.get_loc(year2)] -= top5[year2].sum()
    others_sum['NIT_EXPORTADOR'] = 'Otros'
    others_sum['RAZON_SOCIAL'] = ''
    others_sum['SECTOR_ESTRELLA'] = ''
    others_sum['Participación (%)'] = (others_sum[year2] / total_row[year2].values[0]) * 100
    others_sum['Variación (%)'] = ((others_sum[year2] - others_sum[year1]) / others_sum[year1].replace(0, np.nan)) * 100
    
    # 10. Combinar las filas de top 5, 'Otros' y 'Total' en el DataFrame final
    final_df = pd.concat([top5, others_sum, total_row], ignore_index=True)
    
    # 11. Renombrar las columnas y formatear los valores y porcentajes
    column_names_dict = {
        'NIT_EXPORTADOR': 'NIT',
        'RAZON_SOCIAL': 'Empresa',
        'SECTOR_ESTRELLA': 'Sector',
        year1: f'{transform_year_column_name(year1)} (USD FOB)',
        year2: f'{transform_year_column_name(year2)} (USD FOB)',
        'Variación (%)': 'Variación (%)',
        'Participación (%)': f'Participación {transform_year(year2)} (%)'
    }
    final_df.rename(columns=column_names_dict, inplace=True)
    
    # Formatear los valores y porcentajes
    final_df[f'{transform_year_column_name(year1)} (USD FOB)'] = final_df[f'{transform_year_column_name(year1)} (USD FOB)'].map('{:,.0f}'.format)
    final_df[f'{transform_year_column_name(year2)} (USD FOB)'] = final_df[f'{transform_year_column_name(year2)} (USD FOB)'].map('{:,.0f}'.format)
    final_df['Variación (%)'] = final_df['Variación (%)'].map('{:.1f}%'.format)
    final_df[f'Participación {transform_year(year2)} (%)'] = final_df[f'Participación {transform_year(year2)} (%)'].map('{:.1f}%'.format)

    # Convertir separadores de decimales y miles
    final_df = final_df.astype(str)
    for col in final_df.columns[3:]:
        final_df[col] = final_df[col].apply(lambda x: x.replace(',', 'X').replace('.', ',').replace('X', '.'))
    
    return final_df

In [12]:
def generar_tabla_subsectores(df, year1, year2):
    """
    Genera una tabla de resumen de exportaciones por subsector para dos años específicos,
    incluyendo la diferencia y variación porcentual entre esos años, y filtra el top 5 de subsectores 
    con mayor diferencia.

    Parámetros:
    df (DataFrame): DataFrame con los datos de exportaciones.
    year1 (int): Año inicial para el cálculo de diferencias y variaciones.
    year2 (int): Año final para el cálculo de diferencias y variaciones.

    Pasos del proceso:
    1. Filtrar los datos por los años especificados.
    2. Pivotear los datos para tener los valores de exportación por año y subsector.
    3. Asegurarse de que los años especificados existan en el DataFrame.
    4. Unir las tablas de ambos años.
    5. Calcular los totales por subsector y año.
    6. Calcular la diferencia y la variación porcentual entre los años.
    7. Seleccionar las columnas deseadas para el resumen.
    8. Filtrar el top 5 de subsectores con mayor diferencia.
    9. Renombrar las columnas y formatear los valores y porcentajes.

    Retorna:
    DataFrame: DataFrame con el resumen de exportaciones por subsector.
    """

    # 1. Filtrar los datos por los años especificados
    df_year1 = df[df['YEAR'] == year1].copy()
    df_year2 = df[df['YEAR'] == year2].copy()

    # 2. Pivotear los datos para tener los valores de exportación por año y subsector
    pivot_table_year1 = df_year1.pivot_table(values='VALOR_USD', index='SUBSECTOR', columns='PAIS_DESTINO', aggfunc='sum', fill_value=0)
    pivot_table_year2 = df_year2.pivot_table(values='VALOR_USD', index='SUBSECTOR', columns='PAIS_DESTINO', aggfunc='sum', fill_value=0)

    # 3. Asegurarse de que los años especificados existan en el DataFrame
    pivot_table_year1.columns = [f'{year1}({col})' for col in pivot_table_year1.columns]
    pivot_table_year2.columns = [f'{year2}({col})' for col in pivot_table_year2.columns]

    # 4. Unir las tablas de ambos años
    pivot_table = pivot_table_year1.join(pivot_table_year2, how='outer').fillna(0)
    pivot_table.reset_index(inplace=True)

    # 5. Calcular los totales por subsector y año
    pivot_table[year1] = pivot_table.filter(like=f'{year1}(').sum(axis=1)
    pivot_table[year2] = pivot_table.filter(like=f'{year2}(').sum(axis=1)

    # 6. Calcular la diferencia y la variación porcentual entre los años
    pivot_table['Diferencia'] = pivot_table[year2] - pivot_table[year1]
    pivot_table['Variación (%)'] = ((pivot_table[year2] - pivot_table[year1]) / pivot_table[year1].replace(0, np.nan)) * 100

    # 7. Seleccionar las columnas deseadas para el resumen
    pivot_table = pivot_table[['SUBSECTOR', year1, year2, 'Diferencia', 'Variación (%)']]

    # 8. Filtrar el top 5 de subsectores con mayor diferencia
    pivot_table = pivot_table.nlargest(5, 'Diferencia')

    # 9. Renombrar las columnas y formatear los valores y porcentajes
    column_names_dict = {
        'SUBSECTOR': 'Subsector',
        year1: f'{transform_year_column_name(year1)} (USD FOB)',
        year2: f'{transform_year_column_name(year2)} (USD FOB)',
        'Diferencia': 'Diferencia (USD FOB)',
        'Variación (%)': 'Variación (%)'
    }
    pivot_table.rename(columns=column_names_dict, inplace=True)

    # Formatear los valores y porcentajes
    pivot_table[f'{transform_year_column_name(year1)} (USD FOB)'] = pivot_table[f'{transform_year_column_name(year1)} (USD FOB)'].map('{:,.0f}'.format)
    pivot_table[f'{transform_year_column_name(year2)} (USD FOB)'] = pivot_table[f'{transform_year_column_name(year2)} (USD FOB)'].map('{:,.0f}'.format)
    pivot_table['Diferencia (USD FOB)'] = pivot_table['Diferencia (USD FOB)'].map('{:,.0f}'.format)
    pivot_table['Variación (%)'] = pivot_table['Variación (%)'].map('{:.1f}%'.format)

    # Convertir separadores de decimales y miles
    pivot_table = pivot_table.astype(str)
    for col in pivot_table.columns[1:]:
        pivot_table[col] = pivot_table[col].apply(lambda x: x.replace(',', 'X').replace('.', ',').replace('X', '.'))

    return pivot_table

#### Función para generar lista de tablas

Las siguientes funciones toman como insumo los resultados de getdata y generar_tablas de la sección anterior y generan una lista de tablas para ponerlas en los documentos.

In [13]:
def obtener_datos_exportaciones(session, continentes=None, zonas_geograficas=None, paises=None, departamentos=None, 
                                hubs=None, tlcs=None, tipo_tlcss=None, tipos=None, years_cerrado=None, years_corrido=None, umbral=10000):
    """
    Obtiene los datos de exportaciones totales, número de empresas y datos de empresas para años cerrados y corridos.

    Parámetros:
    session (snowflake.snowpark.Session): Sesión activa en Snowflake.
    continentes (list): Lista de continentes a filtrar.
    zonas_geograficas (list): Lista de zonas geográficas a filtrar.
    paises (list): Lista de países a filtrar.
    departamentos (list): Lista de departamentos a filtrar.
    hubs (list): Lista de hubs a filtrar.
    tlcs (list): Lista de tratados de libre comercio a filtrar.
    tipo_tlcss (list): Lista de tipos de acuerdos comerciales a filtrar.
    tipos (list): Lista de tipos de posición arancelaria a filtrar.
    years_cerrado (list): Lista de años cerrados a filtrar.
    years_corrido (list): Lista de años corridos a filtrar.
    umbral (int): Umbral mínimo de exportación en USD para considerar una empresa.

    Retorna:
    tuple: Ocho DataFrames con los datos de exportaciones y empresas para años cerrados y corridos.
    """
    
    # 1. Obtener datos para año cerrado
    # Exportaciones totales
    df_exportaciones_cerrado = get_data_exportaciones(session, continentes, zonas_geograficas, paises, departamentos, 
                                                      hubs, tlcs, tipo_tlcss, tipos, years_cerrado)
    # Número de empresas
    df_numero_empresas_cerrado = get_data_exportaciones_numero_empresas(session, continentes, zonas_geograficas, paises, departamentos, 
                                                                        hubs, tlcs, tipo_tlcss, tipos, years_cerrado, umbral)
    # Datos de empresas
    df_empresas_cerrado, df_totales_cerrado = get_data_exportaciones_empresas(session, continentes, zonas_geograficas, paises, 
                                                                              departamentos, hubs, tlcs, tipo_tlcss, tipos, years_cerrado)
    
    # 2. Obtener datos para año corrido
    # Exportaciones totales
    df_exportaciones_corrido = get_data_exportaciones(session, continentes, zonas_geograficas, paises, departamentos, 
                                                      hubs, tlcs, tipo_tlcss, tipos, years_corrido)
    # Número de empresas
    df_numero_empresas_corrido = get_data_exportaciones_numero_empresas(session, continentes, zonas_geograficas, paises, departamentos, 
                                                                        hubs, tlcs, tipo_tlcss, tipos, years_corrido, umbral)
    # Datos de empresas
    df_empresas_corrido, df_totales_corrido = get_data_exportaciones_empresas(session, continentes, zonas_geograficas, paises, 
                                                                              departamentos, hubs, tlcs, tipo_tlcss, tipos, years_corrido)
    
    # Devolver los DataFrames generados
    return (df_exportaciones_cerrado, df_numero_empresas_cerrado, df_empresas_cerrado, df_totales_cerrado,
            df_exportaciones_corrido, df_numero_empresas_corrido, df_empresas_corrido, df_totales_corrido)

In [14]:
def generar_todas_tablas_resumen(df, categorias, valores, top_n=None):
    """
    Genera todas las tablas resumen para cada combinación de categorías y valores y asigna nombres descriptivos.

    Parámetros:
    df (DataFrame): DataFrame proveniente de la función get_data_exportaciones().
    categorias (list): Lista de variables categóricas para las cuales se generarán las tablas de resumen.
    valores (list): Lista de variables de valor a agregar ('VALOR_USD' o 'PESO_KG_NETO').

    Retorna:
    tuple: Dos listas de tuplas, cada tupla contiene el nombre de la tabla y el DataFrame con la tabla resumen.
    """

    # Inicializar las listas para almacenar las tablas resumen
    tablas_resumen_usd = []
    tablas_resumen_kg = []

    # Iterar sobre cada categoría y valor
    for categoria in categorias:
        for valor in valores:
            # Generar la tabla resumen
            tabla_resumen = generar_tabla_resumen(df, categoria, valor, top_n)

            # Crear un nombre descriptivo para la tabla
            nombre_tabla = f"RESUMEN_{categoria}_{valor}"

            # Clasificar la tabla en la lista correspondiente
            if valor == 'VALOR_USD':
                tablas_resumen_usd.append((nombre_tabla, tabla_resumen))
            elif valor == 'PESO_KG_NETO':
                tablas_resumen_kg.append((nombre_tabla, tabla_resumen))

    # Retornar las listas de tablas resumen
    return tablas_resumen_usd, tablas_resumen_kg

In [15]:
def generar_listas_tablas_definitivas_exportaciones(session, continentes=None, zonas_geograficas=None, paises=None, departamentos=None, 
                                      hubs=None, tlcs=None, tipo_tlcss=None, tipos=None, years_cerrado=None, 
                                      years_corrido=None, umbral=10000, categorias=None, valores=None, top_n = None):
    """
    Genera las listas de tablas definitivas de exportaciones, tanto en USD como en KG, a partir de las funciones
    obtener_datos_exportaciones, generar_todas_tablas_resumen, generar_tabla_empresas y generar_tabla_subsectores.

    Parámetros:
    session (snowflake.snowpark.Session): Sesión activa en Snowflake.
    continentes (list): Lista de continentes a filtrar.
    zonas_geograficas (list): Lista de zonas geográficas a filtrar.
    paises (list): Lista de países a filtrar.
    departamentos (list): Lista de departamentos a filtrar.
    hubs (list): Lista de hubs a filtrar.
    tlcs (list): Lista de tratados de libre comercio a filtrar.
    tipo_tlcss (list): Lista de tipos de acuerdos comerciales a filtrar.
    tipos (list): Lista de tipos de posición arancelaria a filtrar.
    years_cerrado (list): Lista de años cerrados a filtrar.
    years_corrido (list): Lista de años corridos a filtrar.
    umbral (int): Umbral mínimo de exportación en USD para considerar una empresa.
    categorias (list): Lista de variables categóricas para las cuales se generarán las tablas de resumen.
    valores (list): Lista de variables de valor a agregar ('VALOR_USD' o 'PESO_KG_NETO').
    top_n: Número de categorías top a filtrar. Por defecto no se filtran.
    Retorna:
    dict: Diccionario con las listas de tablas definitivas.
    """

    # Obtener los datos de exportaciones para años cerrados y corridos
    (df_exportaciones_cerrado, df_numero_empresas_cerrado, df_empresas_cerrado, df_totales_cerrado,
     df_exportaciones_corrido, df_numero_empresas_corrido, df_empresas_corrido, df_totales_corrido) = obtener_datos_exportaciones(
        session, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos, years_cerrado, years_corrido, umbral
    )
    
    # Generar todas las tablas resumen para años cerrados
    tablas_resumen_usd_cerrado, tablas_resumen_kg_cerrado = generar_todas_tablas_resumen(df_exportaciones_cerrado, categorias, valores, top_n)

    # Generar todas las tablas resumen para años corridos
    tablas_resumen_usd_corrido, tablas_resumen_kg_corrido = generar_todas_tablas_resumen(df_exportaciones_corrido, categorias, valores, top_n)
    
    # Generar las tablas de empresas para años cerrados y corridos
    tabla_empresas_cerrado = generar_tabla_empresas(df_empresas_cerrado, df_totales_cerrado, years_cerrado[0], years_cerrado[-1])
    tabla_empresas_corrido = generar_tabla_empresas(df_empresas_corrido, df_totales_corrido, years_corrido[0], years_corrido[-1])

    # Generar las tablas de subsectores para años cerrados y corridos
    tabla_subsectores_cerrado = generar_tabla_subsectores(df_exportaciones_cerrado, years_cerrado[0], years_cerrado[-1])
    tabla_subsectores_corrido = generar_tabla_subsectores(df_exportaciones_corrido, years_corrido[0], years_corrido[-1])

    # Crear las listas de resultados
    resumen_usd_cerrado = [(nombre, tabla) for nombre, tabla in tablas_resumen_usd_cerrado]
    resumen_usd_corrido = [(nombre, tabla) for nombre, tabla in tablas_resumen_usd_corrido]
    resumen_kg_cerrado = [(nombre, tabla) for nombre, tabla in tablas_resumen_kg_cerrado]
    resumen_kg_corrido = [(nombre, tabla) for nombre, tabla in tablas_resumen_kg_corrido]

    numero_empresas_cerrado = [("Numero de Empresas Año Cerrado", df_numero_empresas_cerrado)]
    numero_empresas_corrido = [("Numero de Empresas Año Corrido", df_numero_empresas_corrido)]

    tablas_empresas_cerrado = [("Tabla Empresas Año Cerrado", tabla_empresas_cerrado)]
    tablas_empresas_corrido = [("Tabla Empresas Año Corrido", tabla_empresas_corrido)]

    tablas_subsectores_cerrado = [("Tabla Subsectores Año Cerrado", tabla_subsectores_cerrado)]
    tablas_subsectores_corrido = [("Tabla Subsectores Año Corrido", tabla_subsectores_corrido)]

    # Crear un diccionario para organizar los resultados
    resultados = {
        "Resumen USD Cerrado": resumen_usd_cerrado,
        "Resumen USD Corrido": resumen_usd_corrido,
        "Resumen KG Cerrado": resumen_kg_cerrado,
        "Resumen KG Corrido": resumen_kg_corrido,
        "Numero de Empresas Cerrado": numero_empresas_cerrado,
        "Numero de Empresas Corrido": numero_empresas_corrido,
        "Tablas Empresas Cerrado": tablas_empresas_cerrado,
        "Tablas Empresas Corrido": tablas_empresas_corrido,
        "Tablas Subsectores Cerrado": tablas_subsectores_cerrado,
        "Tablas Subsectores Corrido": tablas_subsectores_corrido,
    }

    # Retornar el diccionario con los resultados
    return resultados

In [16]:
# Alemania
continentes = None
zonas_geograficas = None
paises = ['Alemania']
departamentos = None
hubs = None
tlcs = None
tipo_tlcss = None
tipos = ['No Mineras']
years = ['2022', '2023']
years_cerrado = ['2022', '2023']
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
umbral = 10000
categorias = ['TIPO', 'SECTOR', 'DEPARTAMENTO_ORIGEN']
valores = ['VALOR_USD', 'PESO_KG_NETO']
top_n = 5

# Generar las listas de tablas definitivas
resultados = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos, 
    years_cerrado, years_corrido, umbral, categorias, valores, top_n
)

In [17]:
# Acceder a las tablas
resumen_usd_cerrado = resultados["Resumen USD Cerrado"]
resumen_usd_corrido = resultados["Resumen USD Corrido"]
resumen_kg_cerrado = resultados["Resumen KG Cerrado"]
resumen_kg_corrido = resultados["Resumen KG Corrido"]
numero_empresas_cerrado = resultados["Numero de Empresas Cerrado"]
numero_empresas_corrido = resultados["Numero de Empresas Corrido"]
tablas_empresas_cerrado = resultados["Tablas Empresas Cerrado"]
tablas_empresas_corrido = resultados["Tablas Empresas Corrido"]
tablas_subsectores_cerrado = resultados["Tablas Subsectores Cerrado"]
tablas_subsectores_corrido = resultados["Tablas Subsectores Corrido"]

In [18]:
# Ejemplo de impresión de nombres y primeras filas de tablas
for nombre, tabla in resumen_usd_cerrado:
    print(f"Tabla: {nombre}")
    print(tabla.head())

for nombre, tabla in tablas_empresas_cerrado:
    print(f"Tabla: {nombre}")
    print(tabla.head())

Tabla: RESUMEN_TIPO_VALOR_USD
  Tipo de exportación 2022 (USD FOB) 2023 (USD FOB) Variación (%)  \
0  No Mineras          514.520.678    392.791.777    -23,7%         
1  Otros               0              0              nan%           
2  Total               514.520.678    392.791.777    -23,7%         

  Participación 2023 (%)  
0  100,0%                 
1  0,0%                   
2  100,0%                 
Tabla: RESUMEN_SECTOR_VALOR_USD
                                     Sector 2022 (USD FOB) 2023 (USD FOB)  \
0  Café                                      278.686.016    198.697.198     
1  Banano                                    95.834.478     76.420.103      
2  Derivados del café                        23.889.062     22.016.148      
3  Frutas y hortalizas frescas y procesadas  16.827.662     19.801.800      
4  Azúcar                                    13.259.321     13.609.393      

  Variación (%) Participación 2023 (%)  
0  -28,7%        50,6%                  
1  -20,3

## 2. Inversión

## 3. Turismo

## 4. Creación de documento

In [19]:
import pandas as pd
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from docx.enum.style import WD_STYLE_TYPE
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import parse_xml, OxmlElement
from docx.oxml.ns import nsdecls, qn
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL

In [20]:
# Función auxiliar para personalizar estilos
def customize_style(style, font_name, font_size, font_color, bold=False):
    style.font.name = font_name
    style.font.size = font_size
    style.font.color.rgb = font_color
    style.font.bold = bold
    style.paragraph_format.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

In [21]:
def estilos(doc: Document):
    """
    Define y aplica estilos personalizados al documento proporcionado.
    
    Args:
    doc (Document): El documento al que se añadirán los estilos.
    """
  
    # Personalizar el estilo 'Title'
    title_style = doc.styles['Title']
    customize_style(title_style, 'Century Gothic', Pt(16), RGBColor(0, 32, 96), bold=True)

    # Personalizar el estilo 'Heading 1'
    heading1_style = doc.styles['Heading 1']
    customize_style(heading1_style, 'Century Gothic', Pt(14), RGBColor(0, 32, 96), bold=True)

    # Personalizar el estilo 'Heading 2'
    heading2_style = doc.styles['Heading 2']
    customize_style(heading2_style, 'Century Gothic', Pt(12), RGBColor(0, 32, 96), bold=True)

    # Personalizar el estilo 'Heading 3'
    heading3_style = doc.styles['Heading 3']
    customize_style(heading3_style, 'Century Gothic', Pt(12), RGBColor(0, 32, 96), bold=True)

    # Personalizar el estilo 'Normal'
    normal_style = doc.styles['Normal']
    customize_style(normal_style, 'Century Gothic', Pt(11), RGBColor(0, 0, 0))

    # Personalizar el estilo 'Table'
    table_style = doc.styles['Table Grid']
    table_font = table_style.font
    table_font.name = 'Century Gothic'

In [22]:
def add_heading(doc: Document, text: str, level: int, style: str):
    """
    Agrega un encabezado al documento.

    Args:
    doc (Document): El documento al que se añadirá el encabezado.
    text (str): El texto del encabezado.
    level (int): El nivel del encabezado.
    style (str): El estilo del encabezado.
    """
    doc.add_heading(text, level=level).style = doc.styles[style]

In [23]:
def add_paragraph(doc: Document, text: str, style: str):
    """
    Agrega un párrafo al documento.

    Args:
    doc (Document): El documento al que se añadirá el párrafo.
    text (str): El texto del párrafo.
    style (str): El estilo del párrafo.
    """
    p = doc.add_paragraph(text)
    p.style = doc.styles[style]

In [24]:
def set_cell_border(cell, **kwargs):
    """
    Establece los bordes de una celda.
    
    Uso:
    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "val": "single", "color": "#00FF00", "space": "0"},
        left={"sz": 24, "val": "single", "color": "#0000FF", "space": "0"},
        right={"sz": 24, "val": "single", "color": "#000000", "space": "0"},
    )

    Args:
    cell: La celda a la que se aplicarán los bordes.
    kwargs: Un diccionario con las especificaciones para los bordes (tamaño, valor, color y espacio).
    """
    tcPr = cell._element.get_or_add_tcPr()

    # Verificar si ya existen bordes en la celda
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement("w:tcBorders")
        tcPr.append(tcBorders)

    # Aplicar los bordes según las especificaciones proporcionadas
    for edge in ["top", "left", "bottom", "right", "insideH", "insideV"]:
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = "w:{}".format(edge)

            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            for key in ["sz", "val", "color", "space"]:
                if key in edge_data:
                    element.set(qn("w:{}".format(key)), str(edge_data[key]))

In [25]:
def add_table(doc: Document, dataframe: pd.DataFrame, style: str):
    """
    Agrega una tabla al documento a partir de un DataFrame.

    Args:
    doc (Document): El documento al que se añadirá la tabla.
    dataframe (DataFrame): El DataFrame que se convertirá en tabla.
    style (str): El estilo de la tabla.
    """
    if isinstance(dataframe, pd.DataFrame):
        table = doc.add_table(rows=1, cols=len(dataframe.columns))
        table.style = doc.styles[style]
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        table.autofit = True  # Desactivar ajuste automático para controlar el ancho de las celdas

        hdr_cells = table.rows[0].cells
        for i, column in enumerate(dataframe.columns):
            hdr_cells[i].text = str(column)
            hdr_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            hdr_cells[i].vertical_alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            hdr_cells[i].paragraphs[0].runs[0].bold = True
            hdr_cells[i].paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
            hdr_cells[i].paragraphs[0].runs[0].font.size = Pt(10)
            shading_elm = OxmlElement("w:shd")
            shading_elm.set(qn("w:fill"), "#215E99")
            hdr_cells[i]._element.get_or_add_tcPr().append(shading_elm)
            set_cell_border(
                hdr_cells[i],
                top={"sz": 1, "val": "single", "color": "000000"},
                bottom={"sz": 1, "val": "single", "color": "000000"},
                left={"sz": 1, "val": "single", "color": "000000"},
                right={"sz": 1, "val": "single", "color": "000000"},
            )
        
        for index, row in dataframe.iterrows():
            row_cells = table.add_row().cells
            for i, cell in enumerate(row):
                row_cells[i].text = str(cell)
                row_cells[i].paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                row_cells[i].vertical_alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
                row_cells[i].paragraphs[0].runs[0].font.size = Pt(10)
                if index == len(dataframe) - 1:  # Última fila
                    row_cells[i].paragraphs[0].runs[0].bold = True
                    shading_elm = OxmlElement("w:shd")
                    shading_elm.set(qn("w:fill"), "#DAE9F7")
                    row_cells[i]._element.get_or_add_tcPr().append(shading_elm)
                    row_cells[i].paragraphs[0].runs[0].font.color.rgb = RGBColor(0, 0, 0)
                set_cell_border(
                    row_cells[i],
                    top={"sz": 1, "val": "single", "color": "000000"},
                    bottom={"sz": 1, "val": "single", "color": "000000"},
                    left={"sz": 1, "val": "single", "color": "000000"},
                    right={"sz": 1, "val": "single", "color": "000000"},
                )

        # Ajustar el ancho de las celdas para que la tabla ocupe todo el ancho de las márgenes del documento
        for row in table.rows:
            for cell in row.cells:
                cell.width = doc.sections[0].page_width - doc.sections[0].left_margin - doc.sections[0].right_margin

    else:
        print(f"El valor proporcionado no es un DataFrame: {dataframe}")

In [26]:
def agregar_tabla_contenidos(new_doc):
    """
    Agrega una tabla de contenidos al documento proporcionado en el estilo 'Normal'.
    
    Args:
    new_doc (Document): El documento al que se añadirá la tabla de contenidos.
    """
    # Crear un párrafo para el título de la tabla de contenidos y centrarlo
    para = new_doc.add_paragraph("Tabla de Contenidos")
    para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    for run in para.runs:
        run.font.name = 'Century Gothic'
        run.font.size = Pt(11)
        run.bold = True
        run.underline = True
        run.font.color.rgb = RGBColor(0, 32, 96)

    # Crear un párrafo vacío para insertar el campo de la tabla de contenidos
    paragraph = new_doc.add_paragraph()
    paragraph.paragraph_format.space_before = Inches(0)
    paragraph.paragraph_format.space_after = Inches(0)
    run = paragraph.add_run()
    
    # Insertar el campo de la tabla de contenidos
    fldChar = OxmlElement('w:fldChar')  # creates a new element
    fldChar.set(qn('w:fldCharType'), 'begin')  # sets attribute on element
 
    instrText = OxmlElement('w:instrText')
    instrText.set(qn('xml:space'), 'preserve')  # sets attribute on element
    instrText.text = 'TOC \\o "1-3" \\h \\z \\u'   # change 1-3 depending on heading levels you need
 
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'separate')
 
    fldChar3 = OxmlElement('w:t')
    fldChar3.text = "Haga clic derecho para actualizar el campo."
    fldChar3 = OxmlElement('w:updateFields')
    fldChar3.set(qn('w:val'), 'true')
    fldChar2.append(fldChar3)
 
    fldChar4 = OxmlElement('w:fldChar')
    fldChar4.set(qn('w:fldCharType'), 'end')
 
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar4)
 
    p_element = paragraph._p

In [27]:
def add_header_footer(doc: Document, header_image_left: str, header_image_right: str, footer_image: str, footer_text: str):
    """
    Agrega un encabezado con dos imágenes (una a la izquierda y otra a la derecha) y un pie de página con una imagen a la derecha, texto a la izquierda y el número de página en el centro.
    
    Args:
    doc (Document): El documento al que se añadirán el encabezado y el pie de página.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    """
    section = doc.sections[0]

    # Encabezado
    header = section.header
    header_table = header.add_table(rows=1, cols=2, width=doc.sections[0].page_width)
    header_table.autofit = True

    # Imagen izquierda en el encabezado
    header_cell_left = header_table.cell(0, 0)
    header_paragraph_left = header_cell_left.paragraphs[0]
    header_run_left = header_paragraph_left.add_run()
    header_run_left.add_picture(header_image_left, width=Inches(2.0))

    # Imagen derecha en el encabezado
    header_cell_right = header_table.cell(0, 1)
    header_paragraph_right = header_cell_right.paragraphs[0]
    header_paragraph_right.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    header_run_right = header_paragraph_right.add_run()
    header_run_right.add_picture(header_image_right, width=Inches(2.0))

    # Pie de página
    footer = section.footer
    footer_table = footer.add_table(rows=1, cols=3, width=doc.sections[0].page_width)
    footer_table.autofit = True

    # Texto izquierda en el pie de página
    footer_cell_left = footer_table.cell(0, 0)
    footer_paragraph_left = footer_cell_left.paragraphs[0]
    footer_paragraph_left.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    footer_run_left = footer_paragraph_left.add_run(footer_text)
    footer_run_left.font.size = Pt(8)

    # Número de página en el centro del pie de página
    footer_cell_center = footer_table.cell(0, 1)
    footer_paragraph_center = footer_cell_center.paragraphs[0]
    footer_paragraph_center.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    footer_run_center = footer_paragraph_center.add_run()
    fldChar1 = OxmlElement('w:fldChar')
    fldChar1.set(qn('w:fldCharType'), 'begin')
    footer_run_center._r.append(fldChar1)
    instrText = OxmlElement('w:instrText')
    instrText.text = 'PAGE'
    footer_run_center._r.append(instrText)
    fldChar2 = OxmlElement('w:fldChar')
    fldChar2.set(qn('w:fldCharType'), 'end')
    footer_run_center._r.append(fldChar2)
    # Establecer el tamaño de la fuente del número de página
    footer_run_center.font.size = Pt(8)

    # Imagen derecha en el pie de página
    footer_cell_right = footer_table.cell(0, 2)
    footer_paragraph_right = footer_cell_right.paragraphs[0]
    footer_paragraph_right.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    footer_run_right = footer_paragraph_right.add_run()
    footer_run_right.add_picture(footer_image, width=Inches(2.0))

In [28]:
def create_document(resultados, file_path, titulo, fecha, header_image_left, header_image_right, footer_image):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    titulo (str): El título principal del informe.
    fecha (str): La fecha del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES {titulo.upper()}', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(fecha, style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()
        
    # Agregar secciones y tablas
    add_heading(doc, 'Resumen USD Cerrado', level=2, style='Heading 1')
    for nombre, tabla in resultados["Resumen USD Cerrado"]:
        add_heading(doc, nombre, level=3, style='Heading 2')
        add_table(doc, tabla, 'Table Grid')

    # Guardar el documento
    doc.save(file_path)


In [29]:
# Alemania
continentes = None
zonas_geograficas = None
paises = ['Alemania']
departamentos = None
hubs = None
tlcs = None
tipo_tlcss = None
tipos = ['No Mineras']
years = ['2022', '2023']
years_cerrado = ['2022', '2023']
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
umbral = 10000
categorias = ['TIPO', 'SECTOR', 'DEPARTAMENTO_ORIGEN']
valores = ['VALOR_USD', 'PESO_KG_NETO']
top_n = 5

# Generar las listas de tablas definitivas
resultados = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos, 
    years_cerrado, years_corrido, umbral, categorias, valores, top_n
)

In [30]:
# Crear el documento Word
create_document(resultados=resultados, file_path="Informe_Exportaciones.docx",
                titulo="Alemania", fecha="01 de junio de 2023", 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png')

#### CONTINENTES

In [31]:
# Ejemplo con América
# Geografía
continentes = ['América']
zonas_geograficas = None
paises = None
departamentos = None
hubs = None
tlcs = None
tipo_tlcss = None
# Tipos (uno para lo agregado y otro para lo no minero)
tipos_total = ['No Mineras', 'Mineras']
tipos_nme = ['No Mineras']
# Año cerrado (solo varía una vez)
years_cerrado = ['2022', '2023']
# Año corrido (varía todos los meses)
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
# Umbral de 10000 USD para cuenta de empresa
umbral = 10000
# Categorías para tener en cuenta
categorias_total = ['TIPO']
categorias_nme = ['SECTOR', 'SUBSECTOR', 'PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'CADENA', 'ZONA_GEOGRAFICA', 'TLCS']
# Valores a generar tablas
valores = ['VALOR_USD', 'PESO_KG_NETO']
# Número de categorías para top x
top_n_total = None
top_n_nme = 5
# Años para text
year_cerrado = '2023'
year_corrido = 'enero - marzo 2024'
# Fecha para text
fecha_actualizacion = 'MARZO 2024'

In [32]:
# Lista de tablas de totales de exportación
resultados_total = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_total, 
    years_cerrado, years_corrido, umbral, categorias_total, valores, top_n_total)

In [33]:
# Lista de tablas de totales de exportación NME
resultados_nme = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_nme, 
    years_cerrado, years_corrido, umbral, categorias_nme, valores, top_n_nme)

In [34]:
def create_document_continentes(df_total, df_nme, file_path, titulo, fecha, header_image_left, header_image_right, footer_image, year_cerrado, year_corrido):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    titulo (str): El título principal del informe.
    fecha (str): La fecha de actualización del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    year_cerrado (str): Texto con el año cerrado del informe.
    year_corrido (str): Texto con el año corrido del informe.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES CONTINENTES: {str(titulo).upper()}', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(f'ÚLTIMA ACTUALIZACIÓN: {fecha.upper()}', style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()
        

    ###############
    # Exportaciones
    ###############
    add_heading(doc, 'Exportaciones', level=2, style='Heading 1')
    
    #####################
    # Tipo de exportación
    #####################
    add_heading(doc, 'Tipo de exportación', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_total["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_total["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #####################################
    # Exportaciones no minero-energéticas
    #####################################
    add_heading(doc, 'Exportaciones no minero-energéticas', level=2, style='Heading 1')

    ###########
    #  Destinos
    ###########
    add_heading(doc, 'Destinos', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][2][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][2][1]), 'Table Grid')
    doc.add_paragraph()
    
    ########################
    # Departamento de origen
    ########################
    add_heading(doc, 'Departamento de origen', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][3][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][3][1]), 'Table Grid')
    doc.add_paragraph()

    ########
    # Sector
    ########
    add_heading(doc, 'Sector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #############
    # SubSectores
    #############
    add_heading(doc, 'Subsector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    ####################
    # Conteo de Empresas
    ####################
    add_heading(doc, 'Empresas', level=3, style='Heading 2')
    # Año cerrado
    paragraph_cerrado = doc.add_paragraph(f'Número de empresas exportadoras en {str(year_cerrado)}: ', style='Normal')
    run_cerrado = paragraph_cerrado.add_run(f'{str(df_nme["Numero de Empresas Cerrado"][0][1])} empresas')
    run_cerrado.bold = True
    # Año corrido
    paragraph_corrdo = doc.add_paragraph(f'Número de empresas exportadoras a {str(year_corrido)}: ', style='Normal')
    run_corrido = paragraph_corrdo.add_run(f'{str(df_nme["Numero de Empresas Corrido"][0][1])} empresas')
    run_corrido.bold = True

    ################
    # Datos empresas
    ################
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    # Guardar el documento
    doc.save(file_path)

In [35]:
# Crear el documento Word
create_document_continentes(df_total=resultados_total, df_nme=resultados_nme,
                file_path=f"Tres Ejes Continentes - {continentes[0]}.docx",
                titulo=continentes[0], fecha=fecha_actualizacion, 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png', year_cerrado=year_cerrado, year_corrido=year_corrido)

#### HUBS

In [36]:
# Ejemplo con Europa
# Geografía
continentes = None
zonas_geograficas = None
paises = None
departamentos = None
hubs = ['Europa']
tlcs = None
tipo_tlcss = None
# Tipos (uno para lo agregado y otro para lo no minero)
tipos_total = ['No Mineras', 'Mineras']
tipos_nme = ['No Mineras']
# Año cerrado (solo varía una vez)
years_cerrado = ['2022', '2023']
# Año corrido (varía todos los meses)
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
# Umbral de 10000 USD para cuenta de empresa
umbral = 10000
# Categorías para tener en cuenta
categorias_total = ['TIPO']
categorias_nme = ['SECTOR', 'SUBSECTOR', 'PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'CADENA', 'ZONA_GEOGRAFICA', 'TLCS']
# Valores a generar tablas
valores = ['VALOR_USD', 'PESO_KG_NETO']
# Número de categorías para top x
top_n_total = None
top_n_nme = 5
# Años para text
year_cerrado = '2023'
year_corrido = 'enero - marzo 2024'
# Fecha para text
fecha_actualizacion = 'MARZO 2024'

In [37]:
# Lista de tablas de totales de exportación
resultados_total = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_total, 
    years_cerrado, years_corrido, umbral, categorias_total, valores, top_n_total)

In [38]:
# Lista de tablas de totales de exportación NME
resultados_nme = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_nme, 
    years_cerrado, years_corrido, umbral, categorias_nme, valores, top_n_nme)

In [39]:
def create_document_hub(df_total, df_nme, file_path, titulo, fecha, header_image_left, header_image_right, footer_image, year_cerrado, year_corrido):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    titulo (str): El título principal del informe.
    fecha (str): La fecha de actualización del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    year_cerrado (str): Texto con el año cerrado del informe.
    year_corrido (str): Texto con el año corrido del informe.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES HUBs: {str(titulo).upper()}', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(f'ÚLTIMA ACTUALIZACIÓN: {fecha.upper()}', style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()

    ###############
    # Exportaciones
    ###############
    add_heading(doc, 'Exportaciones', level=2, style='Heading 1')
    
    #####################
    # Tipo de exportación
    #####################
    add_heading(doc, 'Tipo de exportación', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_total["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_total["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #####################################
    # Exportaciones no minero-energéticas
    #####################################
    add_heading(doc, 'Exportaciones no minero-energéticas', level=2, style='Heading 1')

    ###########
    #  Destinos
    ###########
    add_heading(doc, 'Destinos', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][2][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][2][1]), 'Table Grid')
    doc.add_paragraph()
    
    ########################
    # Departamento de origen
    ########################
    add_heading(doc, 'Departamento de origen', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][3][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][3][1]), 'Table Grid')
    doc.add_paragraph()

    ########
    # Sector
    ########
    add_heading(doc, 'Sector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #############
    # SubSectores
    #############
    add_heading(doc, 'Subsector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    ####################
    # Conteo de Empresas
    ####################
    add_heading(doc, 'Empresas', level=3, style='Heading 2')
    # Año cerrado
    paragraph_cerrado = doc.add_paragraph(f'Número de empresas exportadoras en {str(year_cerrado)}: ', style='Normal')
    run_cerrado = paragraph_cerrado.add_run(f'{str(df_nme["Numero de Empresas Cerrado"][0][1])} empresas')
    run_cerrado.bold = True
    # Año corrido
    paragraph_corrdo = doc.add_paragraph(f'Número de empresas exportadoras a {str(year_corrido)}: ', style='Normal')
    run_corrido = paragraph_corrdo.add_run(f'{str(df_nme["Numero de Empresas Corrido"][0][1])} empresas')
    run_corrido.bold = True

    ################
    # Datos empresas
    ################
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    # Guardar el documento
    doc.save(file_path)

In [40]:
# Crear el documento Word
create_document_hub(df_total=resultados_total, df_nme=resultados_nme,
                file_path=f"Tres Ejes HUBs - {hubs[0]}.docx",
                titulo=hubs[0], fecha=fecha_actualizacion, 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png', year_cerrado=year_cerrado, year_corrido=year_corrido)

#### PAÍS

In [41]:
# Ejemplo con Alemania
# Geografía
continentes = None
zonas_geograficas = None
paises = ['Alemania']
departamentos = None
hubs = None
tlcs = None
tipo_tlcss = None
# Tipos (uno para lo agregado y otro para lo no minero)
tipos_total = ['No Mineras', 'Mineras']
tipos_nme = ['No Mineras']
# Año cerrado (solo varía una vez)
years_cerrado = ['2022', '2023']
# Año corrido (varía todos los meses)
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
# Umbral de 10000 USD para cuenta de empresa
umbral = 10000
# Categorías para tener en cuenta
categorias_total = ['TIPO']
categorias_nme = ['SECTOR', 'SUBSECTOR', 'PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'CADENA', 'ZONA_GEOGRAFICA', 'TLCS']
# Valores a generar tablas
valores = ['VALOR_USD', 'PESO_KG_NETO']
# Número de categorías para top x
top_n_total = None
top_n_nme = 5
# Años para text
year_cerrado = '2023'
year_corrido = 'enero - marzo 2024'
# Fecha para text
fecha_actualizacion = 'MARZO 2024'

In [42]:
# Lista de tablas de totales de exportación
resultados_total = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_total, 
    years_cerrado, years_corrido, umbral, categorias_total, valores, top_n_total)

In [43]:
# Lista de tablas de totales de exportación NME
resultados_nme = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_nme, 
    years_cerrado, years_corrido, umbral, categorias_nme, valores, top_n_nme)

In [44]:
def create_document_pais(df_total, df_nme, file_path, titulo, fecha, header_image_left, header_image_right, footer_image, year_cerrado, year_corrido):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    titulo (str): El título principal del informe.
    fecha (str): La fecha de actualización del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    year_cerrado (str): Texto con el año cerrado del informe.
    year_corrido (str): Texto con el año corrido del informe.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES PAÍSES: {str(titulo).upper()}', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(f'ÚLTIMA ACTUALIZACIÓN: {fecha.upper()}', style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()

    ###############
    # Exportaciones
    ###############
    add_heading(doc, 'Exportaciones', level=2, style='Heading 1')
    
    #####################
    # Tipo de exportación
    #####################
    add_heading(doc, 'Tipo de exportación', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_total["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_total["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #####################################
    # Exportaciones no minero-energéticas
    #####################################
    add_heading(doc, 'Exportaciones no minero-energéticas', level=2, style='Heading 1')
  
    ########################
    # Departamento de origen
    ########################
    add_heading(doc, 'Departamento de origen', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][3][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][3][1]), 'Table Grid')
    doc.add_paragraph()

    ########
    # Sector
    ########
    add_heading(doc, 'Sector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #############
    # SubSectores
    #############
    add_heading(doc, 'Subsector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    ####################
    # Conteo de Empresas
    ####################
    add_heading(doc, 'Empresas', level=3, style='Heading 2')
    # Año cerrado
    paragraph_cerrado = doc.add_paragraph(f'Número de empresas exportadoras en {str(year_cerrado)}: ', style='Normal')
    run_cerrado = paragraph_cerrado.add_run(f'{str(df_nme["Numero de Empresas Cerrado"][0][1])} empresas')
    run_cerrado.bold = True
    # Año corrido
    paragraph_corrdo = doc.add_paragraph(f'Número de empresas exportadoras a {str(year_corrido)}: ', style='Normal')
    run_corrido = paragraph_corrdo.add_run(f'{str(df_nme["Numero de Empresas Corrido"][0][1])} empresas')
    run_corrido.bold = True

    ################
    # Datos empresas
    ################
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    # Guardar el documento
    doc.save(file_path)

In [45]:
# Crear el documento Word
create_document_pais(df_total=resultados_total, df_nme=resultados_nme,
                file_path=f"Tres Ejes Países - {paises[0]}.docx",
                titulo=paises[0], fecha=fecha_actualizacion, 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png', year_cerrado=year_cerrado, year_corrido=year_corrido)

#### DEPARTAMENTOS

In [46]:
# Ejemplo con Antioquia
# Geografía
continentes = None
zonas_geograficas = None
paises = None
departamentos = ['Antioquia']
hubs = None
tlcs = None
tipo_tlcss = None
# Tipos (uno para lo agregado y otro para lo no minero)
tipos_total = ['No Mineras', 'Mineras']
tipos_nme = ['No Mineras']
# Año cerrado (solo varía una vez)
years_cerrado = ['2022', '2023']
# Año corrido (varía todos los meses)
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
# Umbral de 10000 USD para cuenta de empresa
umbral = 10000
# Categorías para tener en cuenta
categorias_total = ['TIPO']
categorias_nme = ['SECTOR', 'SUBSECTOR', 'PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'CADENA', 'ZONA_GEOGRAFICA', 'TLCS']
# Valores a generar tablas
valores = ['VALOR_USD', 'PESO_KG_NETO']
# Número de categorías para top x
top_n_total = None
top_n_nme = 5
# Años para text
year_cerrado = '2023'
year_corrido = 'enero - marzo 2024'
# Fecha para text
fecha_actualizacion = 'MARZO 2024'

In [47]:
# Lista de tablas de totales de exportación
resultados_total = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_total, 
    years_cerrado, years_corrido, umbral, categorias_total, valores, top_n_total)

In [48]:
# Lista de tablas de totales de exportación NME
resultados_nme = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_nme, 
    years_cerrado, years_corrido, umbral, categorias_nme, valores, top_n_nme)

In [49]:
def create_document_departamento(df_total, df_nme, file_path, titulo, fecha, header_image_left, header_image_right, footer_image, year_cerrado, year_corrido):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    titulo (str): El título principal del informe.
    fecha (str): La fecha de actualización del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    year_cerrado (str): Texto con el año cerrado del informe.
    year_corrido (str): Texto con el año corrido del informe.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES DEPARTAMENTOS: {str(titulo).upper()}', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(f'ÚLTIMA ACTUALIZACIÓN: {fecha.upper()}', style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()

    ###############
    # Exportaciones
    ###############
    add_heading(doc, 'Exportaciones', level=2, style='Heading 1')
    
    #####################
    # Tipo de exportación
    #####################
    add_heading(doc, 'Tipo de exportación', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_total["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_total["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #####################################
    # Exportaciones no minero-energéticas
    #####################################
    add_heading(doc, 'Exportaciones no minero-energéticas', level=2, style='Heading 1')

    ###########
    #  Destinos
    ###########
    add_heading(doc, 'Destinos', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][2][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][2][1]), 'Table Grid')
    doc.add_paragraph()
    
    ########
    # Sector
    ########
    add_heading(doc, 'Sector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #############
    # SubSectores
    #############
    add_heading(doc, 'Subsector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    ######
    # TLCS
    ######
    add_heading(doc, 'Tratados de libre comercio', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][6][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][6][1]), 'Table Grid')
    doc.add_paragraph()

    ####################
    # Conteo de Empresas
    ####################
    add_heading(doc, 'Empresas', level=3, style='Heading 2')
    # Año cerrado
    paragraph_cerrado = doc.add_paragraph(f'Número de empresas exportadoras en {str(year_cerrado)}: ', style='Normal')
    run_cerrado = paragraph_cerrado.add_run(f'{str(df_nme["Numero de Empresas Cerrado"][0][1])} empresas')
    run_cerrado.bold = True
    # Año corrido
    paragraph_corrdo = doc.add_paragraph(f'Número de empresas exportadoras a {str(year_corrido)}: ', style='Normal')
    run_corrido = paragraph_corrdo.add_run(f'{str(df_nme["Numero de Empresas Corrido"][0][1])} empresas')
    run_corrido.bold = True

    ################
    # Datos empresas
    ################
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    # Guardar el documento
    doc.save(file_path)

In [50]:
# Crear el documento Word
create_document_departamento(df_total=resultados_total, df_nme=resultados_nme,
                file_path=f"Tres Ejes Departamentos - {departamentos[0]}.docx",
                titulo=departamentos[0], fecha=fecha_actualizacion, 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png', year_cerrado=year_cerrado, year_corrido=year_corrido)

#### COLOMBIA

In [51]:
# Colombia
# Geografía
continentes = None
zonas_geograficas = None
paises = None
departamentos = None
hubs = None
tlcs = None
tipo_tlcss = None
# Tipos (uno para lo agregado y otro para lo no minero)
tipos_total = ['No Mineras', 'Mineras']
tipos_nme = ['No Mineras']
# Año cerrado (solo varía una vez)
years_cerrado = ['2022', '2023']
# Año corrido (varía todos los meses)
years_corrido = ['2023(Ene-Mar)', '2024(Ene-Mar)']
# Umbral de 10000 USD para cuenta de empresa
umbral = 10000
# Categorías para tener en cuenta
categorias_total = ['TIPO']
categorias_nme = ['SECTOR', 'SUBSECTOR', 'PAIS_DESTINO', 'DEPARTAMENTO_ORIGEN', 'CADENA', 'ZONA_GEOGRAFICA', 'TLCS']
# Valores a generar tablas
valores = ['VALOR_USD', 'PESO_KG_NETO']
# Número de categorías para top x
top_n_total = None
top_n_nme = 5
# Años para text
year_cerrado = '2023'
year_corrido = 'enero - marzo 2024'
# Fecha para text
fecha_actualizacion = 'MARZO 2024'

In [52]:
# Lista de tablas de totales de exportación
resultados_total = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_total, 
    years_cerrado, years_corrido, umbral, categorias_total, valores, top_n_total)

In [53]:
# Lista de tablas de totales de exportación NME
resultados_nme = generar_listas_tablas_definitivas_exportaciones(
    session_activa, continentes, zonas_geograficas, paises, departamentos, hubs, tlcs, tipo_tlcss, tipos_nme, 
    years_cerrado, years_corrido, umbral, categorias_nme, valores, top_n_nme)

In [54]:
def create_document_colombia(df_total, df_nme, file_path, fecha, header_image_left, header_image_right, footer_image, year_cerrado, year_corrido):
    """
    Crea un documento Word con los resultados proporcionados.

    Args:
    resultados (dict): Diccionario con los resultados que se incluirán en el informe.
    file_path (str): Ruta donde se guardará el documento de salida.
    fecha (str): La fecha de actualización del informe.
    header_image_left (str): Ruta de la imagen para el encabezado (izquierda).
    header_image_right (str): Ruta de la imagen para el encabezado (derecha).
    footer_image (str): Ruta de la imagen para el pie de página.
    footer_text (str): Texto para el pie de página.
    year_cerrado (str): Texto con el año cerrado del informe.
    year_corrido (str): Texto con el año corrido del informe.
    """
    doc = Document()
    estilos(doc)

    # Agregar encabezado y pie de página
    footer_text = """Calle 28 # 13ª - 15, Edificio CCI Pisos 35 - 36 | Bogotá, Colombia T: +57 (1) 560 0100 | info@procolombia.co | www.procolombia.co"""

    add_header_footer(doc, header_image_left, header_image_right, footer_image, footer_text)
        
    # Agregar el título principal del informe
    title_paragraph = doc.add_paragraph(f'TRES EJES COLOMBIA', style='Title')
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER

    # Agregar la fecha
    date_paragraph = doc.add_paragraph(f'ÚLTIMA ACTUALIZACIÓN: {fecha.upper()}', style='Title')
    date_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    # Agregar la tabla de contenido
    doc.add_page_break()
    agregar_tabla_contenidos(doc)   
    doc.add_page_break()

    ###############
    # Exportaciones
    ###############
    add_heading(doc, 'Exportaciones', level=2, style='Heading 1')
    
    #####################
    # Tipo de exportación
    #####################
    add_heading(doc, 'Tipo de exportación', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_total["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_total["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #####################################
    # Exportaciones no minero-energéticas
    #####################################
    add_heading(doc, 'Exportaciones no minero-energéticas', level=2, style='Heading 1')

    ###########
    #  Destinos
    ###########
    add_heading(doc, 'Destinos', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][2][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][2][1]), 'Table Grid')
    doc.add_paragraph()
    
    ########################
    # Departamento de origen
    ########################
    add_heading(doc, 'Departamento de origen', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][3][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][3][1]), 'Table Grid')
    doc.add_paragraph()

    ########
    # Sector
    ########
    add_heading(doc, 'Sector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Resumen USD Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    #############
    # SubSectores
    #############
    add_heading(doc, 'Subsector', level=3, style='Heading 2')
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Subsectores Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    ####################
    # Conteo de Empresas
    ####################
    add_heading(doc, 'Empresas', level=3, style='Heading 2')
    # Año cerrado
    paragraph_cerrado = doc.add_paragraph(f'Número de empresas exportadoras en {str(year_cerrado)}: ', style='Normal')
    run_cerrado = paragraph_cerrado.add_run(f'{str(df_nme["Numero de Empresas Cerrado"][0][1])} empresas')
    run_cerrado.bold = True
    # Año corrido
    paragraph_corrdo = doc.add_paragraph(f'Número de empresas exportadoras a {str(year_corrido)}: ', style='Normal')
    run_corrido = paragraph_corrdo.add_run(f'{str(df_nme["Numero de Empresas Corrido"][0][1])} empresas')
    run_corrido.bold = True

    ################
    # Datos empresas
    ################
    # Año cerrado
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Cerrado"][0][1]), 'Table Grid')
    doc.add_paragraph()
    # Año corrido
    add_table(doc, pd.DataFrame(df_nme["Tablas Empresas Corrido"][0][1]), 'Table Grid')
    doc.add_paragraph()

    # Guardar el documento
    doc.save(file_path)

In [56]:
# Crear el documento Word
create_document_colombia(df_total=resultados_total, df_nme=resultados_nme,
                file_path=f"Tres Ejes Colombia.docx", fecha=fecha_actualizacion, 
                header_image_left=r'Insumos/Procolombia/PRO_PRINCIPAL_HORZ_PNG.png', header_image_right=r'Insumos/Logo Ministerio Comercio, Industria y Turismo/Logo MinCit_Mesa de trabajo 1.png',
                footer_image=r'Insumos/Logo Marca País/Logo_MP_EPDLB2.png', year_cerrado=year_cerrado, year_corrido=year_corrido)

In [58]:
resultados["Resumen USD Cerrado"][2][1]

,Departamento de origen,2022 (USD FOB),2023 (USD FOB),Variación (%),Participación 2023 (%)
0,Antioquia,98.489.039,83.145.625,"-15,6%","21,2%"
1,Bogotá,71.828.515,62.664.128,"-12,8%","16,0%"
2,Caldas,76.442.680,56.371.935,"-26,3%","14,4%"
3,Magdalena,69.788.131,43.180.560,"-38,1%","11,0%"
4,Valle del Cauca,19.789.812,36.624.906,"85,1%","9,3%"
5,Otros,178.182.500,110.804.622,"-37,8%","28,2%"
6,Total,514.520.678,392.791.777,"-23,7%","100,0%"


## Final: Cerrar sesión

In [57]:
session_activa.close()